In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [3]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Preprocess

In [4]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                   transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307), (0.3081,))
                   ])),
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, 
                   transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307), (0.3081,))
                   ])),
    batch_size=test_batch_size,
    shuffle=True
)

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Model

In [5]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 20, 5, 1)
    self.conv2 = nn.Conv2d(20, 50, 5, 1)
    self.fc1 = nn.Linear(4*4*50, 500)
    self.fc2 = nn.Linear(500, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2, 2)
    x = x.view(-1, 4*4*50)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)  # F.softmax() + torch.log()

## Optimization

In [6]:
model = Net().to(device)

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

* Parameters 확인
 - Weight, Bias 순서대로 보여줌

In [12]:
params = list(model.parameters())
for param in params:
  print(param.size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


## Before Training

* 학습하기 전 model이 train 할 수 있도록, train mode로 변환

In [13]:
model.train()  # train mode

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

* 모델에 넣을 첫 batch 데이터 추출

In [15]:
data, target = next(iter(train_loader))

In [16]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

* 추출한 batch 데이터를 cpu 또는 gpu와 같은 device에 compile

In [17]:
data, target = data.to(device), target.to(device)

* gradients를 clear하여 새로운 최적화 값을 찾기 위해 준비
* Why do we need to call zero_grad() in PyTorch? - https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch

In [19]:
optimizer.zero_grad()

* 준비한 데이터를 modeldp input으로 넣어 output을 얻음

In [20]:
output = model(data)

* 모델에서 계산한 결과를 loss function에 넣어 정답과 차이 계산

In [21]:
loss = F.nll_loss(output, target)

* Back Propagation을 통해 gradients 계산

In [22]:
loss.backward()

* 계산된 gradient를 parameter에 update

In [24]:
optimizer.step()

## Start Training

In [25]:
epochs = 1
log_interval = 100

In [27]:
for epoch in range(1, epochs+1):
  # Train Mode
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()  # 기울기 계산
    optimizer.step()

    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100 * batch_idx / len(train_loader), loss.item()
      ))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302457
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.230344
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.124192
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.969210
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.591832
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.137470
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.914448
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.683407
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.633837
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.382570


## Evaluation

* Batch Normalization이나 Drop out 같은 layer들을 잠금

In [28]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

* autograd engine off

In [29]:
test_loss = 0
correct = 0

with torch.no_grad():
  data, target = next(iter(test_loader))
  data, target = data.to(device), target.to(device)
  output = model(data)

  test_loss += F.nll_loss(output, target, reduction='sum').item()

  pred = output.argmax(dim=1, keepdim=True)
  correct = pred.eq(target.view_as(pred)).sum().item()

In [30]:
pred

tensor([[4],
        [7],
        [5],
        [4],
        [7],
        [7],
        [6],
        [0],
        [8],
        [5],
        [9],
        [8],
        [7],
        [7],
        [1],
        [8],
        [0],
        [9],
        [5],
        [0],
        [7],
        [6],
        [8],
        [9],
        [8],
        [6],
        [6],
        [7],
        [6],
        [1],
        [4],
        [6],
        [8],
        [7],
        [8],
        [8],
        [8],
        [5],
        [1],
        [4],
        [1],
        [4],
        [8],
        [2],
        [4],
        [9],
        [2],
        [1],
        [6],
        [3],
        [7],
        [1],
        [6],
        [9],
        [2],
        [6],
        [5],
        [8],
        [8],
        [4],
        [2],
        [1],
        [3],
        [6]], device='cuda:0')

In [31]:
test_loss

31.926137924194336

In [33]:
output.shape

torch.Size([64, 10])

In [32]:
pred.shape

torch.Size([64, 1])

In [35]:
target

tensor([4, 7, 5, 4, 7, 7, 6, 0, 8, 5, 4, 1, 7, 2, 1, 8, 0, 9, 5, 0, 7, 6, 8, 7,
        8, 6, 6, 7, 6, 1, 4, 6, 8, 7, 8, 8, 3, 5, 1, 4, 1, 4, 8, 2, 4, 9, 2, 1,
        6, 3, 9, 1, 8, 9, 2, 6, 5, 8, 8, 6, 2, 1, 3, 8], device='cuda:0')

In [34]:
target.shape

torch.Size([64])

In [36]:
target.view_as(pred).shape

torch.Size([64, 1])

In [37]:
pred.eq(target.view_as(pred))

tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True

In [38]:
correct

55

In [39]:
test_loss /= len(test_loader.dataset)

In [40]:
test_loss

0.0031926137924194334

In [43]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)
))


Test set: Average Loss: 0.4893, Accuracy: 8608/10000 (86%)

